Importa varias cosas, request para llamar a HTTPS, JSON para convertir los archivos, IPython.display los muestra para que sean mas legibles y keys agarra lo que esta en los archivos keus.py

In [19]:
import requests
import json
from IPython.display import JSON
from keys import *

Se define la URL, de donde e va a agarrar y las horas, cosas como de donde llega, si esta disponible, si es privado o con cargo, hace la solicitud para la API

In [20]:
#be careful not to run too often as there is a maximum of 200 api calls per month

url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/SAEZ/2025-09-01T08:00/2025-09-01T20:00"

querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}

headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flights_key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

Si el codigo es 200 da el .json, se lee el archivo si existe, agregal os datos que existen y lo guarda, muestra que si se pudo y si no da error de la API

In [21]:
if response.status_code == 200:
    new_data = response.json()

    try:
        with open("data.json", "r") as json_file:
            existing_data = json.load(json_file)
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        existing_data = []

    existing_data.append(new_data)

    with open("data.json", "w") as json_file:
        json.dump(existing_data, json_file, indent=4)
        print("Data appended to data.json file.")
else:
    print("Failed to retrieve data from the API. Status code:", response.status_code)

Data appended to data.json file.


Responde todo de cuando llegan los vuelos, da la hora cuando llegaron, el numero, a donde llegaron, a dodne se van, de que aeropuerto y de donde, que tipo de avion fue y el model y finalmente de que aerolinea fue 

In [24]:
flight_arrivals = response.json()

for flight in flight_arrivals:
    flight_arrivals['arrivals'][0]['arrival']['scheduledTime']
    flight_arrivals['arrivals'][0]['number']
    flight_arrivals['arrivals'][0]['arrival']['terminal']
    flight_arrivals['arrivals'][0]['departure']['airport']['name']
    flight_arrivals['arrivals'][0]['aircraft']['model']
    flight_arrivals['arrivals'][0]['airline']['name']

Toma los vuelos, a que hora esta escrito que va a llegar, el numero de vuelo, de donde, la aerolinea y el avion, hace la tabla de toda la info y muestra la tabla

In [26]:
def flight_extraction(flight):
    return pd.json_normalize({
        'Scheduled_arrival_time': flight['arrival'].get('scheduledTime', nan),
        'flight_number': flight.get('number', nan),
        'from': flight.get('departure', {}).get('airport', {}).get('name', nan),
        'airline': flight.get('airline', {}).get('name', nan),
        'aircraft': flight.get('aircraft', {}).get('model', nan)
    })

flight_arrivals_df = pd.concat(
    [flight_extraction(flight) for flight in flight_arrivals['arrivals']]
)

flight_arrivals_df.head()

,flight_number,from,airline,aircraft,Scheduled_arrival_time.utc,Scheduled_arrival_time.local
0,UX 41,Madrid,Air Europa,Boeing 787-9,2025-09-01 10:35Z,2025-09-01 07:35-03:00
0,AR 1670Q,Buenos Aires,Aerolineas Argentinas,Boeing 737-800,2025-09-01 11:27Z,2025-09-01 08:27-03:00
0,AA 907,Miami,American,Boeing 787-8,2025-09-01 11:55Z,2025-09-01 08:55-03:00
0,LH 510,Frankfurt-am-Main,Lufthansa,Boeing 747-8,2025-09-01 09:25Z,2025-09-01 06:25-03:00
0,AR 1469,Tucumán,Aerolineas Argentinas,Boeing 737-800,2025-09-01 12:45Z,2025-09-01 09:45-03:00
